# Baseline

**Metodologia**: Como baseline, iremos recomendar livros aleatórios.

## 0. Setup

In [1]:
%load_ext autotime

time: 123 µs (started: 2024-02-19 22:52:54 -03:00)


In [2]:
#---- Bibliotecas:

# Análise e manipulação de dados:

import pandas as pd
import numpy as np

# Visualização de dados:

import plotly.io as pio
import plotly.express as px

time: 465 ms (started: 2024-02-19 22:52:54 -03:00)


In [3]:
#---- Data path:

data_path = '../../03-data/'

#---- Template dos gráficos:

pio.templates.default = 'plotly_white'

time: 133 ms (started: 2024-02-19 22:52:54 -03:00)


## 1. Extração dos dados

### 1.1. Dados de treino e teste

In [4]:
df_train = pd.read_parquet(data_path + '02-processed/01-train_data.parquet')
df_test = pd.read_parquet(data_path + '02-processed/02-test_data.parquet')

time: 358 ms (started: 2024-02-19 22:52:54 -03:00)


### 1.2. Full data

In [5]:
df_full = pd.read_parquet(data_path + '02-processed/03-full_data.parquet')

time: 1.02 s (started: 2024-02-19 22:52:55 -03:00)


## 2. Baseline

### 2.1. Separando os livros em uma lista para a escolha aleatória

In [6]:
list_of_books = df_full\
    .groupby('Book-Title', as_index = False)\
    .agg(
        media_rating = ('Book-Rating', 'mean'),
        frequencia = ('ISBN', 'count')
    )\
    .query('frequencia > 10')['Book-Title']\
    .to_list()

time: 1.1 s (started: 2024-02-19 22:52:56 -03:00)


In [7]:
#---- Criando uma função para trazer K itens aleatórios da lista de livros

from random import sample

def choose_random_books(n: int = 5):

    random_book = sample(list_of_books, n)

    return random_book

time: 445 µs (started: 2024-02-19 22:52:57 -03:00)


In [8]:
#---- Criando uma função que:
# - Aplica escolhe K livros aleatórios para cada usuário
# - Cria uma flag para validar se o livro "teste" está dentro das K recomendações
# - Calcula a fórmula da precision

def calculate_precision_k_baseline(df: pd.DataFrame, col_books: str, lst_recommendations: list, k: int = 10):

    df['recommendations'] = df.apply(lambda x: choose_random_books(n = k), axis = 1)

    df['flag_recommendations'] = df.apply(lambda x: x[col_books] in x['recommendations'], axis = 1)

    value_precision_k = df.query('flag_recommendations == 1').shape[0] / df.shape[0] * 100

    precision_k = 0 if value_precision_k == 0 else value_precision_k

    return precision_k

time: 1.58 ms (started: 2024-02-19 22:52:57 -03:00)


In [9]:
precision_k = calculate_precision_k_baseline(df = df_test, col_books = 'Book-Title', lst_recommendations = list_of_books, k = 20)

time: 756 ms (started: 2024-02-19 22:52:57 -03:00)


In [10]:
precision_k

0.0576036866359447

time: 6.21 ms (started: 2024-02-19 22:52:58 -03:00)
